In [3]:
import csv
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService

/Users/tania/Desktop/Master/2S/BDM/P1/BDM/BDM/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [54]:
class RestaurantScrape_GM:

    def __init__(self,driver):
        self.driver = driver

    def wait_for_element(self, locator, timeout=5):
        try:
            element_present = WebDriverWait(self.driver,timeout).until(
                EC.presence_of_element_located(locator)
            )
        except TimeoutException:
            element_present = None
        return element_present

    def accept_cookies(self):

        try:
            # Scroll down to make "Accept all" button visible
            self.driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
            time.sleep(2)  # Wait after scrolling

            # Locate and click "Accept all" button
            WebDriverWait(self.driver, 9).until(EC.element_to_be_clickable((By.XPATH, "//button[.//span[text()='Aceptar todo']]"))).click()

            time.sleep(3)  # Wait for the page to reload

        except Exception as e:
            print("⚠️ No cookie popup detected or failed to click:", e)

    def collect_reviews(self, required_records, url):
        """
        Collect data for the specified number of records
        """
        self.driver.get(url)
        time.sleep(5)

        self.accept_cookies()
        # Search span "More reviews"
        more_reviews_element = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label, 'Reseñas')]"))
            )
        more_reviews_element.click()

        # more_reviews_identifier = (By.XPATH, '//span[@class="wNNZR fontTitleSmall"]')
        # more_reviews_identifier.click()
        time.sleep(5)

        # Locate the scrollable reviews div
        scrollable_div = self.driver.find_element("xpath", '//div[contains(@class, "m6QErb XiKgde")]')

        prev_height = 0
        records = []  # List to store extracted reviews

        while True:
            # Scroll down inside the reviews container
            self.driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
            time.sleep(2)  # Allow time for new reviews to load

            # Get new height after scrolling
            new_height = self.driver.execute_script("return arguments[0].scrollHeight", scrollable_div)

            # Extract visible reviews after scrolling
            review_elements = self.driver.find_elements(By.XPATH, '//*[@class="jftiEf fontBodyMedium "]')

            # Collect review texts and avoid duplicates
            for review in review_elements:
                review_text = review.text.strip()
                if review_text not in records:
                    records.append(review_text)

            # Stop if no new content loads OR we reached required records
            if new_height == prev_height or len(records) >= required_records:
                break

            prev_height = new_height  # Update scroll height
        return records

    def collect_info(self, url):
        self.driver.get(url)
        time.sleep(5)  # Wait for page to load

        self.accept_cookies()

        # Extract Restaurant Name
        try:
            name = self.driver.find_element(By.XPATH, '//h1[contains(@class, "DUwDvf")]').text
        except:
            name = None

        # Extract Rating
        try:
            rating = self.driver.find_element(By.XPATH, '//div[contains(@class, "F7nice ")]//span[@aria-hidden="true"]').text
        except:
            rating = None

        # Extract Address
        try:
            address = self.driver.find_element(By.XPATH, '(//div[contains(@class, "Io6YTe") and contains(@class, "kR99db")])[1]').text
        except:
            address = None

        # Extract Phone Number
        try:
            phone = self.driver.find_element(By.XPATH, '(//div[contains(@class, "Io6YTe") and contains(@class, "kR99db")])[3]').text
        except:
            phone = None

        # Extract Website
        try:
            website = self.driver.find_element(By.XPATH, '(//div[contains(@class, "Io6YTe") and contains(@class, "kR99db")])[2]').text
        except:
            website = None

        # Extract Opening Hours
        try:
            self.driver.find_element(By.XPATH, '//span[contains(@class, "puWIL hKrmvd google-symbols OazX1c")]').click()  # Click to expand hours
            time.sleep(2)
            hours = self.driver.find_element(By.XPATH, '//div[contains(@class, "t39EBf") and contains(@aria-label, "De")]').text
        except:
            hours = None

        self.driver.quit()

        return {
            "Name": name,
            "Rating": rating,
            "Address": address,
            "Phone": phone,
            "Website": website,
            "Opening Hours": hours
        }



In [43]:
chrome_options = Options()
chrome_options.add_argument("--start-maximized")  # Open in full screen
chrome_options.add_argument("--headless")  # Run without opening a window (optional)

chromedriver_path = "/usr/local/bin/chromedriver"
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)



In [55]:
cdriver = RestaurantScrape_GM(driver)

website = "https://www.google.com/maps/search/?api=1&query=Honest+Greens+Pedralbes+Centre"

In [40]:
info = cdriver.collect_info(website)

In [56]:
reviews = cdriver.collect_reviews(10, website)

⚠️ No cookie popup detected or failed to click: Message: 
Stacktrace:
0   chromedriver                        0x00000001079c3808 chromedriver + 6105096
1   chromedriver                        0x00000001079bb40a chromedriver + 6071306
2   chromedriver                        0x0000000107456600 chromedriver + 415232
3   chromedriver                        0x00000001074a82c0 chromedriver + 750272
4   chromedriver                        0x00000001074a8511 chromedriver + 750865
5   chromedriver                        0x00000001074f89c4 chromedriver + 1079748
6   chromedriver                        0x00000001074ce63d chromedriver + 906813
7   chromedriver                        0x00000001074f5c3d chromedriver + 1068093
8   chromedriver                        0x00000001074ce3e3 chromedriver + 906211
9   chromedriver                        0x000000010749a29a chromedriver + 692890
10  chromedriver                        0x000000010749b3f1 chromedriver + 697329
11  chromedriver                   

In [57]:
reviews


[]

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

# 🚀 Setup WebDriver
driver = webdriver.Chrome()

# 🌍 Open Google Maps Page
driver.get("https://www.google.com/maps/search/?api=1&query=L'illa+Diagonal")
time.sleep(5)  # Allow time for the consent page to load

# 🍪 Accept Cookies (Scrolling First)
try:
    # Scroll down to make "Accept all" button visible
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
    time.sleep(2)  # Wait after scrolling

    # Locate and click "Accept all" button
    WebDriverWait(driver, 9).until(EC.element_to_be_clickable((By.XPATH, "//button[.//span[text()='Aceptar todo']]"))).click()
    # accept_button = driver.find_element(By.XPATH, '//button[contains(text(), "Aceptar todo")]')
    # accept_button.click()
    time.sleep(3)  # Wait for the page to reload

    print("✅ Accepted cookies")
except Exception as e:
    print("⚠️ No cookie popup detected or failed to click:", e)

# 🏠 Extract Restaurant Name
try:
    name_element = driver.find_element(By.XPATH, '//h1[contains(@class, "DUwDvf")]')
    name = name_element.text.strip()

    # Alternative: Use `.get_attribute("innerText")` if `.text` fails
    if not name:
        name = name_element.get_attribute("innerText").strip()

    print(f"✅ Restaurant Name: {name}")

except:
    print("❌ Restaurant name not found")

# 🚪 Close Browser
driver.quit()

✅ Accepted cookies
✅ Restaurant Name: L'illa Diagonal
